In [1]:
import pandas as pd
import lxml
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
import warnings

warnings.filterwarnings("ignore")

In [3]:
from lxml import etree
url_list = ["http://feeds.reuters.com/reuters/businessNews",
            "http://feeds.reuters.com/reuters/technologyNews",
            "http://feeds.reuters.com/reuters/sportsNews",
            ]
documents = []

for url in url_list:
    response = requests.get(url)
    xml_page = response.text
    parser = lxml.etree.XMLParser(recover=True, encoding='utf-8')
    documents.append(lxml.etree.fromstring(xml_page.encode("utf-8"), parser=parser))

In [4]:
def print_tag(node):
    print "<%s %s>%s" % (node.tag, " ".join(["%s=%s" % (k,v)for k,v in node.attrib.iteritems()]), node.text)
    for item in node[:25]:
        print "  <%s %s>%s</%s>" % (item.tag, " ".join(["%s=%s" % (k,v)for k,v in item.attrib.iteritems()]), item.text, item.tag)
    print "</%s>" % node.tag

In [5]:
temp_node = documents[0]
print_tag(temp_node)

<rss version=2.0>

  <channel >
</channel>
</rss>


In [6]:
temp_node = temp_node[0]
print_tag(temp_node)

<channel >

  <title >Reuters: Business News</title>
  <link >http://www.reuters.com</link>
  <description >Reuters.com is your source for breaking news, business, financial and investing news, including personal finance and stocks.  Reuters is the leading global provider of news, financial information and technology solutions to the world's media, financial institutions, businesses and individuals.</description>
  <image >
	</image>
  <language >en-us</language>
  <lastBuildDate >Sat, 20 May 2017 21:27:29 -0400</lastBuildDate>
  <copyright >All rights reserved. Users may download and print extracts of content from this website for their own personal and non-commercial use only. Republication or redistribution of Reuters content, including by framing or similar means, is expressly prohibited without the prior written consent of Reuters. Reuters and the Reuters sphere logo are registered trademarks or trademarks of the Reuters group of companies around the world. © Reuters 2017</copyrig

In [7]:
temp_node = temp_node.xpath("item")[0]
print_tag(temp_node)

<item >
		
  <title >U.S., Saudi firms sign tens of billions of dollars of deals as Trump visits</title>
  <description >RIYADH (Reuters) - U.S. and Saudi Arabian companies signed business deals worth tens of billions of dollars on Saturday during a visit by U.S. President Donald Trump, as Riyadh seeks help to develop its economy beyond oil.<div class="feedflare">
<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=oCoKkh6qpTk:_MSXTUxcDgI:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=oCoKkh6qpTk:_MSXTUxcDgI:V_sGLiPBpWU" border="0"></

In [10]:
title_list = []
description_list = []
category_list = []

for xml_doc in documents:
    articles = xml_doc.xpath("//item")
    for article in articles:
        title_list.append(article[0].text)
        description_list.append(article[1].text)
        category_list.append(article[4].text)
        

news_data = pd.DataFrame(title_list, columns=["title"])
news_data["description"] = description_list
news_data["category"] = category_list
print len(news_data)
news_data

30


,title,description,category
0,"U.S., Saudi firms sign tens of billions of dol...",RIYADH (Reuters) - U.S. and Saudi Arabian comp...,businessNews
1,Softbank-Saudi tech fund becomes world's bigge...,RIYADH (Reuters) - The world's largest private...,businessNews
2,GE announces $15 billion of business deals wit...,RIYADH (Reuters) - U.S. technology and enginee...,businessNews
3,"Blackstone, Saudi's PIF plan $40 billion infra...",RIYADH (Reuters) - U.S. private equity firm Bl...,businessNews
4,Softer dollar may be stock silver lining amid ...,NEW YORK (Reuters) - With S&P 500 companies se...,businessNews
5,"With new sheriff in town, South Korea big busi...",SEOUL (Reuters) - A South Korean retail giant ...,businessNews
6,"About 37,000 AT&T workers go on three-day strike","(Reuters) - About 37,000 AT&T Inc workers, or...",businessNews
7,U.S. tax cut hopes sent state collections down...,CHICAGO (Reuters) - U.S. states collected less...,businessNews
8,U.S. regulators open probe into recall of near...,WASHINGTON/SEOUL (Reuters) - U.S. safety regul...,businessNews
9,Trump to offer exclusive contract to service U...,WASHINGTON (Reuters) - President Donald Trump'...,businessNews


In [11]:
news_data["description"].head()

0    RIYADH (Reuters) - U.S. and Saudi Arabian comp...
1    RIYADH (Reuters) - The world's largest private...
2    RIYADH (Reuters) - U.S. technology and enginee...
3    RIYADH (Reuters) - U.S. private equity firm Bl...
4    NEW YORK (Reuters) - With S&P 500 companies se...
Name: description, dtype: object

In [12]:
print news_data["description"][0]

RIYADH (Reuters) - U.S. and Saudi Arabian companies signed business deals worth tens of billions of dollars on Saturday during a visit by U.S. President Donald Trump, as Riyadh seeks help to develop its economy beyond oil.<div class="feedflare">
<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=oCoKkh6qpTk:_MSXTUxcDgI:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=oCoKkh6qpTk:_MSXTUxcDgI:V_sGLiPBpWU" border="0"></img></a>
</div><img src="http://feeds.feedburner.com/~r/reuters/businessNews/~4/oCoKkh6qpTk" height="1" width="1" alt=""/

In [16]:
%%HTML
RIYADH (Reuters) - U.S. and Saudi Arabian companies signed business deals worth tens of billions of dollars on Saturday during a visit by U.S. President Donald Trump, as Riyadh seeks help to develop its economy beyond oil.<div class="feedflare">
<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=oCoKkh6qpTk:_MSXTUxcDgI:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=oCoKkh6qpTk:_MSXTUxcDgI:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=oCoKkh6qpTk:_MSXTUxcDgI:V_sGLiPBpWU" border="0"></img></a>
</div><img src="http://feeds.feedburner.com/~r/reuters/businessNews/~4/oCoKkh6qpTk" height="1" width="1" alt=""/>

In [17]:
news_data["short_description"] = [item[item.find(" - ")+3:item.find("<")] for item in news_data["description"]]
news_data

,title,description,category,short_description
0,"U.S., Saudi firms sign tens of billions of dol...",RIYADH (Reuters) - U.S. and Saudi Arabian comp...,businessNews,U.S. and Saudi Arabian companies signed busine...
1,Softbank-Saudi tech fund becomes world's bigge...,RIYADH (Reuters) - The world's largest private...,businessNews,"The world's largest private equity fund, backe..."
2,GE announces $15 billion of business deals wit...,RIYADH (Reuters) - U.S. technology and enginee...,businessNews,U.S. technology and engineering conglomerate G...
3,"Blackstone, Saudi's PIF plan $40 billion infra...",RIYADH (Reuters) - U.S. private equity firm Bl...,businessNews,U.S. private equity firm Blackstone and Saudi...
4,Softer dollar may be stock silver lining amid ...,NEW YORK (Reuters) - With S&P 500 companies se...,businessNews,With S&P 500 companies set to notch their stro...
5,"With new sheriff in town, South Korea big busi...",SEOUL (Reuters) - A South Korean retail giant ...,businessNews,A South Korean retail giant has shelved contro...
6,"About 37,000 AT&T workers go on three-day strike","(Reuters) - About 37,000 AT&T Inc workers, or...",businessNews,"About 37,000 AT&T Inc workers, or less than 1..."
7,U.S. tax cut hopes sent state collections down...,CHICAGO (Reuters) - U.S. states collected less...,businessNews,U.S. states collected less personal income tax...
8,U.S. regulators open probe into recall of near...,WASHINGTON/SEOUL (Reuters) - U.S. safety regul...,businessNews,U.S. safety regulators have opened a formal in...
9,Trump to offer exclusive contract to service U...,WASHINGTON (Reuters) - President Donald Trump'...,businessNews,President Donald Trump's administration will s...


In [18]:
corpus = news_data["short_description"]
vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus).toarray()
print X.shape
X

(30, 573)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
vectorizer.get_feature_names()[:25]

[u'000',
 u'10',
 u'14',
 u'15',
 u'1980s',
 u'2016',
 u'37',
 u'40',
 u'500',
 u'63',
 u'70',
 u'93',
 u'about',
 u'accessing',
 u'according',
 u'accounts',
 u'acknowledgement',
 u'adding',
 u'administration',
 u'affiliate',
 u'after',
 u'against',
 u'agent',
 u'aggravating',
 u'ago']

In [20]:
categories = news_data["category"].unique()
category_dict = {value:index for index, value in enumerate(categories)}
results = news_data["category"].map(category_dict)
category_dict

{'businessNews': 0, 'sportsNews': 2, 'technologyNews': 1}

In [21]:
print "corpus size: %s" % len(vectorizer.get_feature_names())

corpus size: 573


In [22]:
# Multinomial Naive Bayes Cassifier
x_train,x_test, y_train,y_test = train_test_split(X, results, test_size=0.2, random_state=1, )
clf = MultinomialNB()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.83333333333333337

In [23]:
y_test

17    1
21    2
10    1
19    1
14    1
20    2
Name: category, dtype: int64

In [24]:
clf.predict(x_test)

array([1, 2, 0, 1, 1, 2])

In [25]:
text = ["Who won the Superbawl?"]
vec_text = vectorizer.transform(text).toarray()
category_dict.keys()[category_dict.values().index(clf.predict(vec_text)[0])]

'sportsNews'

In [26]:
text = ["Where is the president"]
vec_text = vectorizer.transform(text).toarray()
category_dict.keys()[category_dict.values().index(clf.predict(vec_text)[0])]

'businessNews'

In [27]:
text = ["season"]
vec_text = vectorizer.transform(text).toarray()
category_dict.keys()[category_dict.values().index(clf.predict(vec_text)[0])]

'sportsNews'

In [28]:
text = ["unemployment"]
vec_text = vectorizer.transform(text).toarray()
category_dict.keys()[category_dict.values().index(clf.predict(vec_text)[0])]

'businessNews'

In [29]:
text = ["defense"]
vec_text = vectorizer.transform(text).toarray()
category_dict.keys()[category_dict.values().index(clf.predict(vec_text)[0])]

'businessNews'

In [30]:
text = ["machine learning"]
vec_text = vectorizer.transform(text).toarray()
category_dict.keys()[category_dict.values().index(clf.predict(vec_text)[0])]

'businessNews'